## DEPENDENCE INSTALL

- Dependencies install for evaluation of metrics
- Instalação das depedendencias necessárias para executar todos as avaliações

In [31]:
%pip install numpy cython
%pip install wheel
%pip install ir_datasets
%pip install scikit-learn
%pip install nltk
%pip install requests
%pip install matplotlib
%pip install pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [32]:
import re
import ir_datasets
import numpy as np
import requests
import json
import nltk
from nltk.tokenize import word_tokenize

In [33]:
dataset = ir_datasets.load('cranfield')

In [34]:
dataset.docs_count()

1400

## INDEXAR DOCUMENTOS

In [44]:
url = 'http://127.0.0.1:8080/nir'
nltk.download("punkt")
count = 0
for doc in dataset.docs_iter():

  sents = list(word_tokenize(doc.text))

  mystr = ' '.join(map(str, sents))

  myobj = {
            'id': doc.doc_id,
            'title': doc.title.strip(),
            'body': mystr.strip()
          }

  response = requests.post(url, json = myobj)
  if response.status_code == 201:
    count += 1
  else:
    print("DOC ID ", doc.doc_id, "Status ", response.status_code)
    print(response.text)

print("Indexed ", count)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thiagoluizrodrigues/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Indexed  1400


In [45]:
dataset.docs_cls()

ir_datasets.datasets.cranfield.CranfieldDoc

## EFETUAR BUSCAS

In [61]:
count = 0
documents = []
for query in dataset.queries_iter():

  # sent = list(word_tokenize(query.text))
  # mystr = ' '.join(map(str, sent))
  qry = "http://127.0.0.1:8080/nir?query=" + query.text + "&semantic=0"
  x = requests.get(qry)

  if x.status_code == 200:
    obj = json.loads(x.text)
    documents.append({
        "query_id":query.query_id,
        "result":obj
    })
    count += 1

print("Total..: ",count)
  # print("Query ID:", query.query_id, "Status code:", x.status_code)

Total..:  225


In [62]:
import functions.documents_process as dp
process = dp.DocumentProcess(dataset, documents,"NIR")
# process.get_query_relevance(query_id=1)

In [63]:
import pandas as pd
import functions.evaluetion_functions as ev
import functions.documents_process as dp

ndcg_metrics = ev.generate_metrics_dcg_k(dataset, documents,"NIR")

df = pd.DataFrame(ndcg_metrics)
df.to_json('nir_ndcg.json')

dp.sum_relevance(ndcg_metrics['nir_ndcg_k']) / \
    len(ndcg_metrics['nir_ndcg_k'])

0.14566947450126058

In [64]:
metrics = ev.recall_precision(dataset, documents, "NIR")
df = pd.DataFrame(metrics)
df.to_json('nir_recall_precision.json')
df

dp.sum_relevance(metrics['f_scores']) / len(metrics['f_scores'])

0.1575940725940726